In [1]:
import json
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
def read_graph(filename):
    with open(filename, 'r') as f:
        # We have the graph encoded as an adjacency list in a JSON file 
        g = json.load(f)
        # The data structure read from JSON is already "good enough" for us
        return g

In [33]:
g = read_graph('wikipediaGraph_simple.json')
n = len(g.keys())


print('Number of nodes: ', n)

Number of nodes:  49142


In [4]:
key_to_pos = dict(zip(g.keys(), range(0,n)))


key_to_pos

{"27/s/_/'s_Hertogenbosch_d3b5.html": 0,
 "27/s/-/Talk~'s-Hertogenbosch_fdcd.html": 1,
 "27/s/-/'s-Hertogenbosch_1463.html": 2,
 "27/n/_/'N_Sync_0bbc.html": 3,
 'o/o/o/User_talk~OOosexy-sparkieoOo_1e21.html': 4,
 'o/o/d/User_talk~Ood_7dfa.html': 5,
 'o/o/r/Oort_cloud.html': 6,
 'o/o/s/Oostende.html': 7,
 'o/o/p/OOP_ce41.html': 8,
 'o/i/l/Image~Oil_Lamp_Christian_Symbol.jpg_ed2d.html': 9,
 'o/i/l/Oil.html': 10,
 'o/i/l/Talk~Oil_1e5e.html': 11,
 'o/i/l/Oil_platform.html': 12,
 'o/i/l/Oil_lamp.html': 13,
 'o/i/l/Oil_well.html': 14,
 'o/i/e/Image~Oies_en_groupe.JPG_a2f5.html': 15,
 'o/i/!/Oi!.html': 16,
 'o/u/i/User_talk~Ouishoebean_1733.html': 17,
 'o/u/u/User_talk~Ouuplas_cb26.html': 18,
 'o/u/d/Image~Oude_Haarlemmermeer.jpg_5fdd.html': 19,
 'o/u/l/Image~Oulu_coatofarms.svg_8672.html': 20,
 'o/u/l/Oulu.html': 21,
 'o/u/l/Image~Oulu_Rantakatu_2006_05_16.JPG_c448.html': 22,
 'o/u/a/Ouaddaï.html': 23,
 'o/u/r/Our_Universe_7181.html': 24,
 'o/u/r/Image~Ours_brun_parcanimalierpyrenees_2.jpg_6

In [5]:
row = []
col = []
data = []

for key in g.keys():
    for edge in g[key]:
        row.append(key_to_pos[key])
        col.append(key_to_pos[edge])
        data.append(1/len(g[key]))
        
        
mat = csr_matrix((data, (row, col)), shape=(n, n))

In [6]:
a = mat.dot(np.ones(n))

sum(a == 1)

14508

In [32]:
def compute_R(graph):
    n = len(graph.keys())
    # we make a dictionary saving for each key in the graph
    # the corresponding index in the matrix
    key_to_pos = dict(zip(graph.keys(), range(0,n)))
    row = []
    col = []
    data = []

    for key in g.keys():
        for edge in g[key]:
            row.append(key_to_pos[key])
            col.append(key_to_pos[edge])
            data.append(1/len(g[key]))
    R = csr_matrix((data, (row, col)), shape=(n, n))
    
    return R

In [42]:
def PageRank_iteration(x, R, J, alpha):
    n = len(x)
    P = (1 - alpha) * R
    x_prime = (P.T).dot(x) + alpha * J
    return x_prime

In [43]:
def compute_PageRank(graph, alpha, epsilon):
    n = len(graph.keys())
    # We compute the transition matrix without the teleportation
    R = compute_R(graph)
    # The jump vector is imply a vector of ones divided by its length
    J = np.ones(n)/n
    # The starting point can be a uniform distribution across all nodes
    # x = np.ones(n)/n
    # ...or a random stochastic vector
    x = np.random.rand(n)
    x = x/x.sum()
    # We can now iterate until the norm one of the changes in the
    # last iteration goes below epsilon
    err = np.inf # initially infinity
    while (err > epsilon):
        x_new = PageRank_iteration(x, R, J, alpha)
        err = (abs(x_new - x)).sum()
        print(err)
        x = x_new
    print("PageRank scores:")
    print(x.shape)
    dictionary = {}
    for i, k in enumerate(graph.keys()):
        #print(f"{k}: {x[0,i]}")
        dictionary[k] = x[i]
    return x, dictionary

In [47]:
x, d = compute_PageRank(g, 0.1, 0.001)

1.3327620469985153
0.5990622799152815
0.2024156612640305
0.06787624055880093
0.027143456935648805
0.012659829616234207
0.006221520023289405
0.003218270806647274
0.001759673260306674
0.0010517924166833898
0.0006995316157667597
PageRank scores:
(49142,)


In [48]:
for w in sorted(d, key=d.get, reverse=True):
    if 'User' not in w and 'Wiki' not in w and 'Help' not in w and 'Image' not in w:
        print(w, d[w])

c/a/t/Special~Categories_101d.html 0.01345646864333563
g/n/u/GNU_Free_Documentation_License_9d01.html 0.0065179869953776925
u/n/i/United_States_09d4.html 0.0022425868043349953
e/n/g/English_language.html 0.0021989252466322984
g/f/d/GFDL_75b9.html 0.001685863667791453
l/a/w/Law.html 0.0015180755866310747
e/n/c/Encyclopedia.html 0.0010080831186846345
i/n/t/Internet_Protocol_7ac7.html 0.0009743412530472346
b/o/o/Book.html 0.0008014914505259054
o/p/e/Open_content.html 0.0007817996575713396
m/o/n/Money.html 0.0007816406288380457
f/i/l/Film.html 0.0007480821516640918
c/o/p/Copyleft.html 0.0007299527654039442
f/r/e/French_language.html 0.0007294742169354009
2/0/0/2001.html 0.000711139272380194
g/e/r/German_language.html 0.0007110733669819079
d/i/c/Dictionary.html 0.0006717345682934263
r/u/l/Rule.html 0.0006709280242304423
s/o/n/Song.html 0.0006569509700390552
n/o/n/Non-profit.html 0.0006518402659861693
w/e/b/Website.html 0.0006459701309527916
m/o/v/Movie.html 0.0006439703263591564
f/a/i/Fair_

s/e/a/Season.html 1.985835696098855e-05
h/i/m/Himalaya.html 1.985517925068227e-05
i/n/f/Infection.html 1.9848038180755856e-05
1/6/5/1650s.html 1.9841202657953843e-05
h/a/m/Hamburg.html 1.9834790933871823e-05
e/l/i/Elizabeth_I_of_England_2ce6.html 1.9826992541735598e-05
w/a/t/Water_Deer_30cf.html 1.981952780760486e-05
1/9/0/1901.html 1.9816342352571423e-05
h/e/a/Category~Health_95b9.html 1.980794907798645e-05
a/r/i/Aristotle.html 1.9761060929821927e-05
m/a/r/Mars_(planet).html 1.975470373956852e-05
u/2E/s/U.S._postal_abbreviations_d7c6.html 1.974046727528844e-05
s/h/i/Shiva.html 1.972819960663449e-05
n/e/t/Net_jargon.html 1.9719822989132544e-05
d/e/s/Dessert.html 1.9717198422771955e-05
f/i/n/Finance.html 1.9716323230558217e-05
l/i/s/List_of_U.S._states_by_elevation_e223.html 1.970798026705032e-05
s/e/r/Service.html 1.969365477286548e-05
f/o/r/Forest.html 1.968792687885751e-05
n/a/p/Napoleon_Bonaparte_f92d.html 1.967963640833607e-05
1/9/1/1913.html 1.96608100558706e-05
1/6/6/1660.html 1.

f/o/u/Four.html 1.0779425171721907e-05
a/r/t/Art_film.html 1.0771876889845835e-05
l/o/c/Local_area_network.html 1.077136094253482e-05
t/r/a/Transparency.html 1.076662060035001e-05
g/e/n/Genocide.html 1.0757350249108612e-05
v/i/l/Vilnius.html 1.0756415822088923e-05
d/i/c/Dictatorship.html 1.0740600162514679e-05
s/t/u/Stuntman.html 1.073821560845533e-05
b/r/i/Brittany_Murphy_babd.html 1.0737146767730975e-05
t/o/k/Tokelau.html 1.0732042425717375e-05
n/o/r/Norfolk_Island_312a.html 1.0731921782194053e-05
c/o/l/Columbus.html 1.0725239580910637e-05
n/b/c/NBC_89b7.html 1.0719535353589384e-05
p/r/o/WP~PROTECT_6d34.html 1.0718271844235773e-05
f/e/r/Fertilization.html 1.0714366475494141e-05
j/2E/_/J._M._W._Turner_f57b.html 1.0710901478988562e-05
d/p/_/WP~DP_58bd.html 1.070984024801048e-05
g/r/e/Great_Fire_Of_London_f6fe.html 1.0709142489938966e-05
i/n/d/Category~India_948e.html 1.070589074099996e-05
s/o/n/Sony.html 1.070502906746113e-05
1/0/6/1066.html 1.0704665545206558e-05
h/i/s/History_of_slav

p/i/n/Pink_Floyd_f331.html 7.219331383193828e-06
s/e/r/Sermon.html 7.21914336813426e-06
g/r/e/Gregor_Mendel_b43a.html 7.218777961880696e-06
c/u/p/Cup.html 7.218446484036775e-06
l/u/s/Lusatian_Neisse_b9a5.html 7.217430014321214e-06
w/e/s/Westminster_Abbey_ded8.html 7.213671833985649e-06
r/o/m/Romance.html 7.212458922358907e-06
n/u/m/Category~Number_theory_ef8a.html 7.212418608408332e-06
g/o/u/Gouda_(cheese).html 7.211214163939466e-06
n/o/r/North_Vietnam_5079.html 7.20974018716356e-06
e/q/u/Equidae.html 7.209176805867576e-06
o/l/i/Olive_(disambiguation).html 7.2080067720903994e-06
f/i/g/Fight.html 7.204310289509347e-06
t/o/r/Toronto.html 7.200586063655346e-06
o/r/g/Organic_compound.html 7.199338814278135e-06
t/w/e/Twelve_Apostles_56fc.html 7.198480146482604e-06
a/r/t/Article_(grammar).html 7.197347779968936e-06
s/h/i/Shirt.html 7.195822826016991e-06
s/t/o/Stopwatch.html 7.1932482549708166e-06
g/o/t/Gotthold_Ephraim_Lessing_9e89.html 7.192325790190029e-06
v/i/c/Vicente_Piccio,_Jr._b73f.ht

i/n/t/Interpretation.html 5.3494164659385285e-06
r/o/m/Category~Roman_gods_and_goddesses_9158.html 5.349317933864827e-06
q/u/o/Quotient.html 5.3482347406967914e-06
c/r/a/Crater.html 5.3477528035935175e-06
x/e/n/Xenarthra.html 5.3475584948989195e-06
h/s/i/Hsinchu_County_01cf.html 5.347400519101352e-06
1/3/4/1340.html 5.347201305469196e-06
s/u/r/Surface_area.html 5.347019062834088e-06
o/d/d/Odd_number.html 5.346550457962783e-06
n/e/o/Neoptolemus.html 5.346507454024901e-06
b/r/o/Brown_versus_board_of_education.html 5.3465035277186864e-06
w/r/i/Category~Writing_1185.html 5.346347939044356e-06
2/0/0/Category~2003.html 5.345649823382018e-06
a/l/a/Alan_Alda_35e3.html 5.345461752708304e-06
c/o/s/Cost_of_living.html 5.344847448105382e-06
g/a/n/Gangtok.html 5.3447191178949475e-06
s/l/e/Sleeve.html 5.344500085669821e-06
t/u/r/Category~Turkish_football_clubs_9a35.html 5.344453765221771e-06
n/o/r/North_Africa_6a9e.html 5.34246361305701e-06
p/s/y/Psychiatry.html 5.341881462392129e-06
m/u/g/Mughal_Em

...

l/i/b/Libertarian.html 2.493475427691515e-06
e/n/d/Template~Endspoiler_7e79.html 2.493406104334304e-06
b/i/r/Template~Birthyr_bac5.html 2.493406104332811e-06
d/e/a/Template~Deathyr_c277.html 2.4934061043313778e-06
d/a/b/Template~Dablink_fdf2.html 2.4934061043266115e-06
l/i/s/Talk~List_of_cat_breeds_8800.html 2.493339661612466e-06
7/1/0/Category~710s.html 2.4932928345681787e-06
f/r/a/Talk~Francis_Bacon_bba5.html 2.4931947444043406e-06
c/o/l/Collingwood.html 2.493188402336836e-06
a/f/l/AFL_01f4.html 2.493188402336836e-06
c/o/m/Category~Compositions_by_Dmitri_Shostakovich_3d41.html 2.492990946446036e-06
k/c/_/KC_&_the_Sunshine_Band_2f76.html 2.4929562415272476e-06
j/u/r/Category~Jurassic_cfb8.html 2.4929408823178417e-06
b/l/o/Blouse.html 2.4925340929159707e-06
c/a/r/Carbohydrates.html 2.4924486423021795e-06
e/t/h/Ethanol_fuel.html 2.49242389501762e-06
b/u/b/Talk~Bubonic_plague_26f7.html 2.492332750265482e-06
u/2E/s/U.S._President_8806.html 2.492209749496729e-06
1/1/7/Category~1175.html 2.

p/l/e/Pleiades.html 2.3719534065390286e-06
p/h/o/Phoebe.html 2.371953406539028e-06
a/b/i/Abiword.html 2.3716579651622906e-06
3/_/d/3_December_dad1.html 2.3715857463043047e-06
m/a/g/Magnesia.html 2.3714633590773543e-06
1/6/5/Category~1651.html 2.3714136239300166e-06
m/a/u/Talk~Mauna_Loa_c9e5.html 2.371412871526505e-06
n/i/h/Nihilism.html 2.3712396683794675e-06
f/o/r/Template~Former_F1_driver_f620.html 2.371215396024175e-06
1/3/2/Category~1326.html 2.371181821233365e-06
t/r/a/Transcaucasus.html 2.37117662730974e-06
a/b/s/Absolute_monarchy.html 2.3711437004137144e-06
a/p/a/Talk~Apartheid_4806.html 2.371033631781841e-06
v/a/g/Vaginal_secretion.html 2.3708698324707117e-06
t/h/e/The_Good_Life_1ce8.html 2.3708217548069253e-06
h/e/t/Heterodontosaurus.html 2.370742488186011e-06
p/s/y/Psychedelic_Pop_b5d9.html 2.3707309631217913e-06
s/o/n/Sonic_boom.html 2.370633524780155e-06
1/1/9/Category~1198.html 2.3700216074938375e-06
l/e/v/Levant.html 2.3698510841379495e-06
e/d/u/Category~Educators_39b2.ht

f/e/n/Fenerbahçe.html 2.120392694944058e-06
h/u/m/Human_Death_dd4e.html 2.119910819006692e-06
2/9/_/29_February_e894.html 2.1195020621872445e-06
8/_/n/8_November_88da.html 2.1194247814417006e-06
f/a/l/Falun_Dafa_3572.html 2.119193418716581e-06
j/u/n/Junctions.html 2.1191286914716218e-06
i/n/t/Intrusive_vulcanicity.html 2.1188508572982544e-06
a/m/e/American_Constitution_f91f.html 2.1188508572982544e-06
e/m/i/Emit.html 2.1184862533092476e-06
r/o/l/Roller_coasters.html 2.118234939584047e-06
a/s/s/Assault_Rifle_0e07.html 2.1179569889755348e-06
s/a/v/Savannah.html 2.117943943369354e-06
s/e/p/September_11,_2001.html 2.117341382884925e-06
e/m/p/Emperor_Akihito_of_Japan_0a00.html 2.1170727322393145e-06
o/r/g/Organization_for_Economic_Co-operation_and_Development_ffad.html 2.1167388900290353e-06
d/i/p/Dipavali.html 2.1167388900290353e-06
z/a/r/Zarathushtra.html 2.1167388900290353e-06
s/h/a/Shankaracharya.html 2.1167388900290353e-06
s/ã/o/São_Paulo_(city)_cfca.html 2.115944736761765e-06
m/a/j/Ma

c/o/r/Cornwallis.html 2.0349192137072157e-06
c/o/r/Coral_Sea_Islands_Territory_a1cc.html 2.0349192137072157e-06
c/o/r/Coruña.html 2.0349192137072157e-06
c/o/r/Coral_mushrooms.html 2.0349192137072157e-06
c/o/r/Cormoros_Islands_3fbb.html 2.0349192137072157e-06
c/o/r/Corse.html 2.0349192137072157e-06
c/o/r/Cordoba,_Argentina_3591.html 2.0349192137072157e-06
c/o/t/Cototient.html 2.0349192137072157e-06
c/o/s/Cost_Of_Living_1dac.html 2.0349192137072157e-06
c/o/n/Confucian.html 2.0349192137072157e-06
c/o/n/Convection_rain.html 2.0349192137072157e-06
c/o/n/Concert-master.html 2.0349192137072157e-06
c/o/n/Control_Paradigm_3680.html 2.0349192137072157e-06
c/o/n/Conceptual_Metaphor_f79d.html 2.0349192137072157e-06
c/o/n/Containers.html 2.0349192137072157e-06
c/o/n/Concertmistress.html 2.0349192137072157e-06
c/o/n/Consistancy.html 2.0349192137072157e-06
c/o/n/Contaminated.html 2.0349192137072157e-06
c/o/n/Contaminates.html 2.0349192137072157e-06
c/o/n/Template~Continent_5cd0.html 2.034919213707215

m/o/t/Mother_Tongue_2adb.html 2.0349192137072157e-06
m/o/t/Mothers.html 2.0349192137072157e-06
m/o/t/Motel.html 2.0349192137072157e-06
m/o/e/Moebius_strip.html 2.0349192137072157e-06
m/o/e/Moebius_band.html 2.0349192137072157e-06
m/o/s/Mosovce.html 2.0349192137072157e-06
m/o/s/Moselle_river.html 2.0349192137072157e-06
m/o/s/Mosel.html 2.0349192137072157e-06
m/o/s/Moschus_fuscus.html 2.0349192137072157e-06
m/o/s/Moschus_berezovskii.html 2.0349192137072157e-06
m/o/s/Moses_(Michelangelo_50da.html 2.0349192137072157e-06
m/o/s/Mosses.html 2.0349192137072157e-06
m/o/s/Moschus_moschiferus.html 2.0349192137072157e-06
m/o/s/Moschidae.html 2.0349192137072157e-06
m/o/s/Moss_city.html 2.0349192137072157e-06
m/o/s/Moschus_chrysogaster.html 2.0349192137072157e-06
m/o/n/Monica_Lewinsky_scandal_628b.html 2.0349192137072157e-06
m/o/n/Monotremes.html 2.0349192137072157e-06
m/o/n/Monkeys.html 2.0349192137072157e-06
m/o/n/Monicagate.html 2.0349192137072157e-06
m/o/n/Mongolian_Death_Worm_9efe.html 2.034919

In [49]:
sum(x)

0.9918124565482772

In [47]:
g['u/n/i/University_of_Chicago_7f0f.html']

['m/o/t/Motto.html',
 'l/a/t/Latin.html',
 '1/8/9/1890.html',
 'u/n/i/University_President_71fa.html',
 'e/m/p/Employee.html',
 'u/n/d/Undergraduate.html',
 'p/o/s/Postgraduate_education.html',
 'c/h/i/Chicago.html',
 'i/l/l/Illinois.html',
 'u/n/i/United_States_09d4.html',
 'u/r/b/Urban_area.html',
 'm/a/s/Mascot.html',
 'w/e/b/Website.html',
 'u/n/i/University.html',
 'c/h/i/Chicago.html',
 's/t/u/Wikipedia~Stub_72af.html',
 'c/a/t/Special~Categories_101d.html',
 'c/o/l/Category~Colleges_and_universities_30f2.html',
 'u/n/i/University_of_Chicago_7f0f.html',
 'c/o/m/Wikipedia~Community_Portal_6a3c.html',
 's/i/m/Wikipedia~Simple_talk_51fd.html',
 'c/o/n/Help~Contents_22de.html',
 'j/a/n/User~JAnDbot_5c44.html',
 't/h/i/User~Thijs!bot_bdf4.html',
 't/b/c/User~TBC_96ba.html',
 'm/e/r/User~Meriga_30ea.html',
 'z/e/p/User~Zephyr2k_5b2b.html',
 'f/l/a/User~FlaBot_747f.html',
 'y/u/r/User~YurikBot_b393.html',
 't/e/x/Wikipedia~Text_of_the_GNU_Free_Documentation_License_702a.html',
 'c/o/p/W

In [12]:
contents = read_graph('wikipediaGraph_simple_contents.json')

In [62]:
topic = input('Enter your topic: ')

n = len(contents.keys())
keys_topic={}

for idx, key in enumerate(contents.keys()):
    lower_key_content = [x.lower() for x in contents[key]]
    if topic in lower_key_content:
        keys_topic[key] = contents[key] 

Enter your topic: italy


In [72]:
s = generate_seed(g, contents)

Enter your topic: history


In [76]:
generate_seed('europe', g, contents)

Enter your topic: europe


array([0., 0., 0., ..., 0., 0., 0.])

In [54]:
def generate_seed(topic, g, contents):    

    n = len(contents.keys())
    key_to_pos = dict(zip(g.keys(), range(0,n)))
    seeds = np.zeros(n)

    for key in contents.keys():
        lower_key_content = [x.lower() for x in contents[key]]
        if topic in lower_key_content:
            seeds[key_to_pos[key]] = 1

    seeds = seeds/len(seeds)
    return seeds

In [150]:
def compute_PersonalizedPageRank(graph, topic, contents, alpha, epsilon):
    n = len(graph.keys())
    # We compute the transition matrix without the teleportation
    R = compute_R(graph)
    # The jump vector is imply a vector of ones divided by its length
    J_s = generate_seed(topic, graph, contents)
    #J = np.ones(n)/n
    # The starting point can be a uniform distribution across all nodes
    # x = np.ones(n)/n
    # ...or a random stochastic vector
    x = np.random.rand(n)
    x = x/x.sum()
    # We can now iterate until the norm one of the changes in the
    # last iteration goes below epsilon
    err = np.inf # initially infinity
    while (err > epsilon):
        x_new = PageRank_iteration(x, R, J_s, alpha)
        err = (abs(x_new - x)).sum()
        print(err)
        x = x_new
    #print("PageRank scores:")
    #print(x.shape)
    dictionary = {}
    for i, k in enumerate(graph.keys()):
        #print(f"{k}: {x[0,i]}")
        dictionary[k] = x[i]
        
    
    return x, dictionary

In [157]:
topic = input('Insert the topic: ')

x, d = compute_PersonalizedPageRank(g, topic, contents, 0.3, 0.0001)

Insert the topic: wall
1.2782877732331925
0.5155385397651647
0.19709871470019052
0.10825437227951572
0.07207999142294878
0.050252880418697424
0.03512851653890629
0.024555161505470872
0.01716404419246726
0.011997448677970698
0.008386026744896516
0.005861678289585638
0.004097199551173401
0.0028638602413244307
0.00200177973522344
0.0013992025340464156
0.0009780132963802329
0.000683610690849673
0.0004778293992910873
0.0003339925952366139
0.00023345370623257893
0.00016317915511376006
0.00011405873929865148
7.972461648982864e-05


In [158]:
for w in sorted(d, key=d.get, reverse=True):
    not_useful_pages= ['User', 'Category', 'Wiki','Help','Image','Special','License', 'GFDL']
    mask = [x not in w for x in not_useful_pages]
    if all(mask):
        print(w, d[w])


w/a/l/Wall.html 8.874425527835011e-06
b/r/u/Bruntal.html 7.023030669860231e-06
b/r/i/Brick.html 6.695116097472955e-06
f/e/n/Fence.html 6.560580650637854e-06
c/o/l/Column.html 6.510165813618481e-06
t/i/l/Tile.html 6.445055035268111e-06
d/e/f/Defensive_wall.html 6.380144094159175e-06
m/o/v/Movie_screen.html 6.3584682779915675e-06
d/a/m/Dam.html 6.34337698113868e-06
d/o/o/Doorstop.html 6.340213973168997e-06
p/e/t/Pet_door.html 6.339042090571239e-06
g/a/t/Gate.html 6.319513688668447e-06
p/o/s/Poster.html 6.292758323455046e-06
r/i/g/Right_angle.html 6.292709657415066e-06
d/o/o/Door.html 8.777174426448929e-07
e/n/g/English_language.html 8.113473268056366e-07
u/n/i/United_States_09d4.html 8.099175780201381e-07
c/i/t/City.html 7.918667293643756e-07
f/l/o/Floor.html 6.779557217978423e-07
l/a/w/Law.html 6.13394323474677e-07
f/i/l/Film.html 5.066920338994819e-07
t/h/e/Theater.html 4.898543089911967e-07
g/r/e/Great_Wall_of_China_a878.html 4.42341877107727e-07
p/a/p/Paper.html 4.0904529080791335e-0

d/o/l/Dollar.html 5.089234400685287e-09
m/a/r/March_25.html 5.086052777615002e-09
b/a/r/Barcelona.html 5.0850233256232766e-09
d/a/l/Dallas,_Texas_2045.html 5.084917144763683e-09
e/a/s/East_Asia_cb20.html 5.084248665182525e-09
h/o/m/Homer.html 5.078800760459412e-09
c/a/r/Carcassonne.html 5.075613390714786e-09
l/o/s/Los_Angeles_983d.html 5.073686909229009e-09
c/o/m/Communist.html 5.073083473397071e-09
n/o/b/Nobel_Prize_c061.html 5.069972114560548e-09
1/9/2/1922.html 5.068614195786197e-09
j/o/b/Job.html 5.0637933583117666e-09
a/r/t/Art_store.html 5.061140935124466e-09
m/o/b/Mobile_telephone.html 5.060410005453811e-09
i/n/t/International_Phonetic_Alphabet_96f8.html 5.057803535817674e-09
d/w/a/Dwarf_planet.html 5.055426281882991e-09
i/n/t/Integrity.html 5.041561815123394e-09
p/o/s/Postage_stamp.html 5.040671311835697e-09
1/9/2/1929.html 5.037120396858983e-09
n/e/w/New_Delhi_b3e7.html 5.0340107755547325e-09
a/b/i/Abide_with_me.html 5.0338275186853626e-09
f/e/b/February_1.html 5.0333128737999

a/m/e/America_Online_d3b0.html 2.1485902048967446e-09
s/t/o/Stone_Age_b753.html 2.1471895209465385e-09
g/a/e/Gaelic.html 2.1470501051376353e-09
w/o/o/Wood_grain.html 2.1464586639080825e-09
1/8/0/1801.html 2.145132938343312e-09
p/r/o/Propulsion_tools.html 2.1446410231719334e-09
1/8/8/1883.html 2.143632690732379e-09
f/u/e/Fuel.html 2.1430228606193947e-09
c/o/n/Conservative.html 2.142634826962677e-09
p/r/i/Prime_Minister_of_the_United_Kingdom_dd87.html 2.14220182969493e-09
a/s/c/ASCII_art_8901.html 2.141396498084706e-09
d/i/a/Diarrhea.html 2.1411061214381705e-09
o/i/l/Oil_lamp.html 2.1396049619097802e-09
c/o/n/Constantinople.html 2.13942279836164e-09
s/u/i/Suicide.html 2.1392170160619262e-09
i/n/t/Intestine.html 2.137849512504199e-09
j/a/m/James_K._Polk_65bc.html 2.1374089545910923e-09
s/e/s/Sessile.html 2.13619180452605e-09
1/0/s/10s.html 2.1331958340852973e-09
c/o/p/Talk~Copyright_0511.html 2.1331439500326794e-09
c/o/r/Corn.html 2.13088612761637e-09
t/u/b/Tuba.html 2.130752683172919e-09

s/u/p/Super_Nintendo_Entertainment_System_5602.html 9.71869458084619e-10
f/r/a/Frankfurt.html 9.707913674848926e-10
o/f/f/Offspring.html 9.70714102266756e-10
u/n/i/Talk~University_95be.html 9.704627285792382e-10
a/b/s/Absolute_zero.html 9.703100121752207e-10
c/a/m/Camelid.html 9.701088411175382e-10
q/u/e/Quebec_City,_Quebec_5605.html 9.698151270032908e-10
h/a/m/Talk~Hamas_7d89.html 9.695667594724623e-10
a/d/_/AD_e182.html 9.687414681772013e-10
n/i/e/Niece.html 9.68237926916731e-10
a/l/a/Alanis_Morissette_39bf.html 9.681150160784731e-10
n/b/c/NBC_89b7.html 9.677890912586243e-10
m/s/-/MS-DOS_bae8.html 9.671307954878592e-10
c/h/a/Charles_Kay_Ogden_380b.html 9.670247157849306e-10
v/i/t/Talk~Vitamin_D_e333.html 9.665372670448053e-10
t/y/r/Tyr.html 9.664513765097905e-10
k/o/a/Koala.html 9.661875310011449e-10
p/h/a/Pharaoh.html 9.659621691905678e-10
u/n/i/University_of_Georgia_bdea.html 9.654665698978186e-10
l/y/n/Lynyrd_Skynyrd_5f78.html 9.65350776407866e-10
a/n/c/Ancient_India_1454.html 9.6

n/a/y/Nayarit.html 5.090396940663754e-10
c/e/n/Central_Europe_1cd5.html 5.089349498235372e-10
2/1/_/21_September_e9c9.html 5.084098787402488e-10
r/o/m/Romance.html 5.083813088817918e-10
h/i/l/Hillary_Clinton_c91c.html 5.082381798924675e-10
n/i/k/Nike.html 5.079325775133698e-10
c/l/i/Clint_Eastwood_469b.html 5.078522427868426e-10
1/7/4/1743.html 5.077711169080014e-10
i/n/u/Template~Inuse_2e6a.html 5.077149144522966e-10
a/d/v/Adventures_of_Huckleberry_Finn_7388.html 5.076757416055016e-10
9/5/5/955.html 5.076623201868646e-10
m/u/h/Muhammad_Ali_9e91.html 5.075151682490841e-10
m/a/t/Matanzas_province.html 5.073543789457577e-10
d/e/v/Talk~Devon_ccb4.html 5.073429024955902e-10
w/i/n/Winnipeg.html 5.072908153852178e-10
f/r/e/Frenulum.html 5.072002484914719e-10
c/h/u/Talk~Church_5ab5.html 5.069504693344362e-10
s/o/u/Southern_United_States_764b.html 5.068236245147061e-10
m/o/s/Mosquito.html 5.067428198540571e-10
w/i/l/Will.html 5.067145867337536e-10
v/i/e/Vienna_Convention_on_the_Law_of_Treaties

...

o/s/m/Osmium.html 7.699707672377281e-11
m/i/l/Millenium.html 7.699170121444192e-11
s/a/n/Sango.html 7.698976259489413e-11
t/h/e/The_Last_Supper_(painting)_d6de.html 7.695277244962441e-11
1/9/6/1964_Summer_Olympics_c346.html 7.693125303951816e-11
f/i/a/Fiat.html 7.691254424989092e-11
a/i/n/Ainu.html 7.690244250969405e-11
k/ö/b/Köbes.html 7.685684889140793e-11
l/a/w/Lawrence's_Goldfinch_b9aa.html 7.685589827777502e-11
g/r/a/Grasses.html 7.684617448225324e-11
m/o/u/Moustache.html 7.684518689284972e-11
h/e/i/Heinrich-Heine-University_e28f.html 7.684445160731813e-11
l/e/o/Leonardo_of_Pisa_aedb.html 7.684196769847039e-11
d/i/r/Dirk_Nowitzki_4f5b.html 7.684071912153206e-11
3/0/_/30_January_99e7.html 7.680953939076882e-11
a/u/b/Aubergine.html 7.680817685395947e-11
s/i/v/Sivaya_Subramuniyaswami_1898.html 7.680500071031659e-11
w/i/l/William_Randolph_Hearst_c64a.html 7.678770723275865e-11
a/s/t/Asthma.html 7.678061091875236e-11
g/a/r/Gargoyle.html 7.67162574816698e-11
h/i/s/Talk~History_of_Austra

w/a/d/Wadaiko.html 3.587711494626867e-11
u/p/m/Upminster.html 3.58672328845686e-11
k/l/o/Klondike_Gold_Rush_7dcd.html 3.5859710405217474e-11
h/o/l/Holden.html 3.5835523725747787e-11
a/l/2E/List_of_people_by_name~_Al..._98cb.html 3.582158755332881e-11
z/i/m/Zimbabwe_dollar.html 3.58215063231531e-11
d/o/w/Down's_syndrome.html 3.5814261665385585e-11
b/a/p/Baptised.html 3.58139353597155e-11
p/a/u/Paul_Karrer_3395.html 3.580230364889997e-11
a/d/_/Ad_hominem.html 3.5780986085092017e-11
1/2E/_/1._FC_Nürnberg_a3c8.html 3.576852863878889e-11
l/o/n/London_Borough_of_Hackney_2929.html 3.574309774215844e-11
d/o/u/Douala.html 3.5741870067229664e-11
f/o/o/Talk~Food_chain_dee8.html 3.568763327522003e-11
a/m/i/Amiens.html 3.5666863593655425e-11
f/e/n/Fennel.html 3.558902415076287e-11
s/o/u/South_Cadbury_ceda.html 3.557685654086885e-11
2/6/_/26_January_d5ff.html 3.557097816134482e-11
e/i/n/Eindhoven.html 3.5516809138259e-11
4/7/7/477.html 3.550561053402306e-11
h/e/a/Headstreams.html 3.54393940343795e-1

h/i/l/Hildburghausen.html 1.1409484386697705e-11
w/h/a/WHAM!_61fb.html 1.1401733116738494e-11
a/s/h/Ashford.html 1.1397986013854823e-11
p/o/r/Port_Hedland,_Western_Australia_eca1.html 1.1379079932165029e-11
i/n/s/Instant_noodles.html 1.1357100254811598e-11
j/e/n/Jenna_Jameson_fd39.html 1.134137554735334e-11
s/i/l/Silicon_chip.html 1.1339005762555239e-11
t/w/i/Twiggy.html 1.1334044796309991e-11
k/y/l/Kyle_Busch_b22f.html 1.1333580222995952e-11
t/e/s/Testes.html 1.1314478618934804e-11
n/e/u/Talk~Neutrino_0d7a.html 1.1284103344275174e-11
f/c/_/FC_Bayern_München_6658.html 1.1271538952069793e-11
m/o/n/Montgomery.html 1.1263394548672036e-11
u/s/_/US_Army_6d69.html 1.1248226125110485e-11
m/a/l/Template~Malaysia_95c1.html 1.1235204095522989e-11
d/e/p/Dependencies.html 1.1225000395314179e-11
s/t/a/Talk~Star_Trek_3cc1.html 1.121647726932186e-11
h/y/d/Talk~Hydroelectricity_23ef.html 1.1216269607734127e-11
a/n/a/Talk~Anaesthesia_b19a.html 1.121374821770196e-11
c/a/r/Cartoons.html 1.120193237739070

i/p/i/Ipiau.html 0.0
i/p/_/IP_addresses_3529.html 0.0
u/i/m/UiMiyazaki_1d51.html 0.0
u/l/l/Ullans.html 0.0
u/l/a/Ulan_Ude_73a7.html 0.0
u/l/r/Ulrich_Friedrich_Wilhelm_Joachim_Ribbentrop_2061.html 0.0
u/l/r/Ulrich_Friedrich_Wilhelm_Joachim_von_Ribbentrop_aa90.html 0.0
u/l/t/Ultraviolet_rays.html 0.0
u/l/t/Ultraviolet_ray.html 0.0
u/2E/_/U._S._Army_671a.html 0.0
u/2E/_/U._S._Congress_c5a6.html 0.0
u/2E/s/U.S._Army_6c52.html 0.0
u/2E/s/U.S._invasion_of_Afghanistan_dfd5.html 0.0
u/2E/s/U.S.A._a0be.html 0.0
u/2E/s/U.s._dollar.html 0.0
u/k/i/UKIP_14f9.html 0.0
u/k/_/UK_Independence_Party_ae82.html 0.0
u/k/r/Ukrainian_Hryvnia_d629.html 0.0
u/g/l/Ugly_Duckling_9ab5.html 0.0
u/g/a/Uga.html 0.0
u/r/o/Urodela.html 0.0
u/r/l/Talk~URL_0375.html 0.0
u/r/a/Urania.html 0.0
u/r/a/Uralic_language.html 0.0
u/r/s/Ursus_malayanus.html 0.0
u/r/s/Ursus_maritimus.html 0.0
u/t/e/Uterine_tube.html 0.0
u/s/_/US_Yard_343b.html 0.0
u/s/_/US_states_923c.html 0.0
u/s/_/Us_dollar.html 0.0
u/s/_/US_Cup_64d9.html 0.0
u

h/e/i/Heilonjiang.html 0.0
h/e/l/Helarctos_malayanus.html 0.0
h/e/l/Helen_(mythology).html 0.0
h/e/x/Hexahedron.html 0.0
h/e/c/Hector_(mythology).html 0.0
h/e/c/Hectolitre.html 0.0
h/e/c/Hectolittre.html 0.0
h/e/a/Heart_transplantation.html 0.0
h/e/r/Hermaphrodites.html 0.0
h/e/r/Herbs.html 0.0
h/e/r/Hernan_Cortes_ffbb.html 0.0
h/e/t/Heteroptera.html 0.0
h/e/t/Talk~Heterosexual_8ab2.html 0.0
h/e/y/Hey_Arnold_bed2.html 0.0
h/e/m/Hematopoietin.html 0.0
h/e/m/Hemiptera.html 0.0
h/e/m/Hemimetabolous.html 0.0
h/e/m/Hemopoietin.html 0.0
h/s/v/HSV_6878.html 0.0
h/à/_/Hà_Nội_dbd5.html 0.0
h/y/d/Hydropotes_inermis.html 0.0
h/y/d/Hydrogen_Peroxide_2814.html 0.0
h/y/d/Hydropotes_inermis_argyropus.html 0.0
h/y/d/Hydropotes.html 0.0
h/y/d/Hydropotinae.html 0.0
h/y/d/Hydropotes_inermis_inermis.html 0.0
h/y/a/Hyaeninae.html 0.0
h/y/a/Hyaena_brunnea.html 0.0
h/y/a/Hyaena_hyaena.html 0.0
h/y/e/Hyenas.html 0.0
h/y/p/Hypertext_Transfer_Protocol_fbc3.html 0.0
h/y/p/Hyper_text_markup_language.html 0.0
h/y/